# 处理丢失数据

有两种丢失数据：
- None
- np.nan(NaN)

## 1. None

None是Python自带的，其类型为python object。因此，None不能参与到任何计算中。

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [2]:
n1 = np.array([1,2,None])
n2 = np.array([1,2,np.nan])

In [4]:
#  'int' and 'NoneType'   None是python的类型  不能进行运算
# n1.sum()

In [5]:
#nan not a number可以进行计算
n2.sum()

nan

In [10]:
df = DataFrame({'Python':[88,104,113],'数学':[118,132,119]},
               columns = ['Python','数学','英语'])
df['英语'] = [80,None,90]
df

,Python,数学,英语
0,88,118,80.0
1,104,132,NaN
2,113,119,90.0


In [11]:
df.mean(axis = 0)

Python    101.666667
数学        123.000000
英语         85.000000
dtype: float64

object类型的运算要比int类型的运算慢得多  
计算不同数据类型求和时间  
%timeit np.arange(1e5,dtype=xxx).sum()

In [7]:
%timeit np.arange(1e5,dtype=int).sum()

132 µs ± 7.83 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
%timeit np.arange(1e5,dtype=float).sum()

166 µs ± 7.75 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%timeit np.arange(1e5,dtype=object).sum()

8.61 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 2. np.nan（NaN）

np.nan是浮点类型，能参与到计算中。但计算的结果总是NaN。

但可以使用np.nan*()函数来计算nan，此时视nan为0。

In [13]:
np.nan + 9

nan

## 3. pandas中的None与NaN

### 1) pandas中None与np.nan都视作np.nan

创建DataFrame

In [14]:
df = DataFrame(np.random.randint(0,150,size=(3,4)),columns=['python','java','php','c'])
df

,python,java,php,c
0,41,64,57,12
1,78,23,131,11
2,111,60,75,146


使用DataFrame行索引与列索引修改DataFrame数据

In [23]:
# 只能使用 行索引和列索引修改none数据
df.loc[0,'c'] = None
df.loc[1,'php'] = np.nan
df

,python,java,php,c
0,41,64,57.0,NaN
1,78,23,NaN,11.0
2,111,60,75.0,146.0


练习:观察ndarray和Series分别赋予None的区别

In [24]:
n = np.array([1,2,3,None])
n

array([1, 2, 3, None], dtype=object)

In [25]:
s = Series(n)
s

0       1
1       2
2       3
3    None
dtype: object

In [26]:
s2 = Series(np.array([1,2,3,np.nan]))
s2

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

### 2) pandas中None与np.nan的操作

- ``isnull()``
- ``notnull()``
- ``dropna()``: 过滤丢失数据
- ``fillna()``: 填充丢失数据

(1)判断函数
- ``isnull()``
- ``notnull()``

In [60]:
df

,语文,数学,英语,python
张三,40.0,25.0,NaN,17.0
李四,29.0,46.0,43.0,46.0


In [39]:
# 是否是Nan值
df.isnull()

,语文,数学,英语,python
张三,False,False,True,False
李四,False,False,False,False


In [61]:
# 只要列中有一个nan的值  整列都返回true
df.isnull().any()   # 默认有参数axis = 0  是列

语文        False
数学        False
英语         True
python    False
dtype: bool

In [41]:
# 按行来判断如果有一个nan的值  整行都返回true
df.isnull().any(axis=1)   

张三     True
李四    False
dtype: bool

In [42]:
# 只有一列全市nan的值  返回true
df.isnull().all()

语文        False
数学        False
英语        False
python    False
dtype: bool

In [43]:
#notnull(),all(axis = 1) 配合索引，就可以获取不为空的数据
# 意味着把行中有nan的行过滤掉
df[df.notnull().all(axis = 1)]

,语文,数学,英语,python
李四,29,46,43.0,46


(2) 过滤函数
- ``dropna()``

可以选择过滤的是行还是列（默认为行）

In [44]:
df.dropna(axis = 1)

,语文,数学,python
张三,40,25,17
李四,29,46,46


In [45]:
df

,语文,数学,英语,python
张三,40,25,NaN,17
李四,29,46,43.0,46


也可以选择过滤的方式 how = 'all'

In [46]:
df.loc[1] = np.nan
df

,语文,数学,英语,python
张三,40.0,25.0,NaN,17.0
李四,29.0,46.0,43.0,46.0
1,NaN,NaN,NaN,NaN


In [49]:
df.dropna(how = 'all',inplace=True)  # inplace 是否替换原DataFrame
df

,语文,数学,英语,python
张三,40.0,25.0,NaN,17.0
李四,29.0,46.0,43.0,46.0


(3) 填充函数 Series/DataFrame
- ``fillna()``

In [53]:
# 以50来填充nan的值
df.fillna(value=50)

,语文,数学,英语,python
张三,40.0,25.0,50.0,17.0
李四,29.0,46.0,43.0,46.0


可以选择前向填充还是后向填充

In [56]:
# ffill 是向选择向前填充  取前面的值填充  bfill是取后面的值填充  
df.fillna(method='bfill')

,语文,数学,英语,python
张三,40.0,25.0,43.0,17.0
李四,29.0,46.0,43.0,46.0


对于DataFrame来说，还要选择填充的轴axis。记住，对于DataFrame来说：

- axis=0：index/行
- axis=1：columns/列

In [59]:
# 指定以列来为参照物   默认axis=0,取前面一行来填充   
df.fillna(method='ffill',axis=1)

,语文,数学,英语,python
张三,40.0,25.0,25.0,17.0
李四,29.0,46.0,43.0,46.0


============================================

练习7：

1. 简述None与NaN的区别

2. 假设张三李四参加模拟考试，但张三因为突然想明白人生放弃了英语考试，因此记为None，请据此创建一个DataFrame,命名为ddd3

3. 老师决定根据用数学的分数填充张三的英语成绩，如何实现？
    用李四的英语成绩填充张三的英语成绩？

============================================

In [35]:
index = ['张三','李四']
columns = ['语文','数学','英语','python']
data = np.random.randint(0,50,size=(2,4))
df = DataFrame(data=data,index=index,columns=columns)
df

,语文,数学,英语,python
张三,40,25,32,17
李四,29,46,43,46


In [36]:
df.loc['张三','英语'] = None
df

,语文,数学,英语,python
张三,40,25,NaN,17
李四,29,46,43.0,46


In [37]:
# 用前面的数据填充后面的nan值
df.fillna(method='ffill',axis=1)

,语文,数学,英语,python
张三,40.0,25.0,25.0,17.0
李四,29.0,46.0,43.0,46.0
